In [203]:
import requests
import time
import pandas as pd

# import a token
from config import *

In [204]:
v = 5.131
count_fetching_posts = 100
count_fetching_comments = 50
depth = 10
sleep_time = 0.4

In [207]:
def get_user_id(link):
    id = link
    if 'vk.com/' in link:
        id = link.split('/')[-1]
    if not id.replace('id', '').isdigit():
        id = requests.get(f'https://api.vk.com/method/utils.resolveScreenName?&v={v}&screen_name={id}&access_token={token}').json()['response']['object_id']
    else:
        id = id.replace('id', '')
    return int(id)

user_id = get_user_id(url_for_search_user)
user_id

22122222

In [208]:
subscription_ids = requests.get(f'https://api.vk.com/method/users.getSubscriptions?&v={v}&user_id={user_id}&access_token={token}').json()["response"]['groups']['items']

In [209]:
groups_list = ['-' + str(id) for id in subscription_ids]
group_param = ', '.join(str(id) for id in groups_list)

newsfeed = requests.get(f'https://api.vk.com/method/newsfeed.get?&v={v}&filters=post&source_ids={group_param}&count={count_fetching_posts}&access_token={token}').json()['response']
posts_from_subscriptions = newsfeed['items']

In [210]:
for i in range(depth - 1):
    next_from = newsfeed['next_from']
    newsfeed = requests.get(f'https://api.vk.com/method/newsfeed.get?&v={v}&start_from={next_from}&filters=post&source_ids={group_param}&count={count_fetching_posts}&access_token={token}').json()['response']
    posts_from_subscriptions += newsfeed['items']
    time.sleep(sleep_time)
len(posts_from_subscriptions)

1000

In [211]:
liked_posts = []
temp_posts = []
liked_api = ""
limit = 1

In [212]:
def update_liked_posts():
    global liked_api, limit, liked_posts, temp_posts
    if liked_api == "":
        return
    url = 'https://api.vk.com/method/execute?access_token={}&v={}&code=return%20[{}];'.format(token, v, liked_api)
    response = requests.get(url).json()['response']
    is_liked_list = [x for x in response if x != False]
    for i in range(len(is_liked_list)):
        if is_liked_list[i]["liked"] == 1:
            liked_posts.append(temp_posts[i])
    limit = 1
    liked_api = ''
    temp_posts = []
    time.sleep(sleep_time)

for post in posts_from_subscriptions:
    item_id = post['post_id']
    owner_id = post['source_id']
    temp_posts.append(post)
    liked_api += 'API.likes.isLiked({{\'user_id\':{},\'type\':\'post\',\'owner_id\':{},\'item_id\':{}}}),'.format(user_id, owner_id, item_id)
    limit = limit + 1
    if limit == 25:
        update_liked_posts()
update_liked_posts()

In [213]:
user_wall_posts = requests.get(f'https://api.vk.com/method/wall.get?&v={v}&owner_id={user_id}&count={count_fetching_posts}&access_token={token}').json()["response"]['items']

In [214]:
created_by_user_wall_posts = []
reposted_user_wall_posts = []
for post in user_wall_posts:
    if "copy_history" in post:
        reposted_user_wall_posts.append(post)
    else:
        created_by_user_wall_posts.append(post)

In [215]:
all_posts = liked_posts + user_wall_posts
all_comments = []
comment_api = ""
limit = 1

In [216]:
def update_comments():
    global comment_api, all_comments, limit
    if comment_api == "":
        return
    url = 'https://api.vk.com/method/execute?access_token={}&v={}&code=return%20[{}];'.format(token, v, comment_api)
    response = requests.get(url).json()['response']
    comment_hub = [x for x in response if x != False]
    for comment_branch in comment_hub:
        for commentary in comment_branch['items']:
            all_comments.append(commentary)
    limit = 1
    comment_api = ''
    time.sleep(sleep_time)

for post in all_posts:
    try:
        post_id = post['post_id']
    except Exception:
        post_id = post['id']
    owner_id = post['owner_id']
    comment_api += 'API.wall.getComments({{\'type\':\'post\',\'count\':{},\'owner_id\':{},\'post_id\':{}}}),'.format(count_fetching_comments, owner_id, post_id)
    limit = limit + 1
    if limit == 25:
        update_comments()
update_comments()

In [217]:
user_comments = []
for comment in all_comments:
    if comment["from_id"] == user_id:
        user_comments.append(comment)

In [218]:
pd.DataFrame(liked_posts).to_csv("parsed_data/liked_posts.csv")
pd.DataFrame(created_by_user_wall_posts).to_csv("parsed_data/created_posts.csv")
pd.DataFrame(reposted_user_wall_posts).to_csv("parsed_data/reposted_posts.csv")
pd.DataFrame(user_comments).to_csv("parsed_data/comments.csv")